In [2]:
import os
import json
import pandas as pd
import traceback

In [3]:
import langchain
print(langchain.__version__)

0.3.23


In [4]:
from langchain.chat_models import ChatOpenAI


In [5]:
from dotenv import  load_dotenv
load_dotenv()

True

In [6]:
llm = ChatOpenAI( openai_api_key= os.getenv("OPENAI_API_KEY") ,
    model_name="gpt-3.5-turbo",
    temperature=0.5,
   
)

C:\Users\Suraj Yadav\AppData\Local\Temp\ipykernel_33320\1455916755.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI( openai_api_key= os.getenv("OPENAI_API_KEY") ,


In [8]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [9]:
RESPONSE_JSON = {
    "1":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer",
    },
    "2":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer",
    },
    "3":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer",
    }
}

In [10]:
TEMPLATE="""
TEXT: {text}
You are a expert MCQ generator.Given the above text, it is your job to create a quiz of{number} multiple choice questions for {subject} students in {tone} tone. Make sure the question are not repeated and check all the questions to be conforming the text as well. Make sure to format your response like RESPONSE_JSON below adn use it as a guide. \
Ensure to make {number} MCQs.
### RESPONSE_JSON:
{response_json}

"""

In [11]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE,
)

In [12]:
quiz_chain= LLMChain(
    llm=llm,
    prompt=quiz_generation_prompt,
    verbose=True,
    output_key="quiz",
)

C:\Users\Suraj Yadav\AppData\Local\Temp\ipykernel_33320\49273341.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain= LLMChain(


In [14]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a multiple choice quiz for {subject} students. \
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity. \
if the quiz is not at per with the cognitive and analytical abilities of the students, \
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the students ability.
Quiz_MCQ: {quiz}

Check from an expert English writer of the above quiz:
"""

In [15]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE2,
)

In [16]:
review_chain= LLMChain(
    llm=llm,
    prompt=quiz_evaluation_prompt,
    verbose=True,
    output_key="review",
)

In [17]:
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=["text", "number", "subject", "tone", "response_json"],
    output_variables=["quiz", "review"],
    verbose=True,
)

In [18]:
file_path=r"C:\Users\Suraj Yadav\workspace\personal-projects\mcqgen\data.txt"

In [19]:
file_path

'C:\\Users\\Suraj Yadav\\workspace\\personal-projects\\mcqgen\\data.txt'

In [20]:
with open(file_path, "r") as file:
    TEXT=file.read()
print(TEXT)

Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] The application of ML to business problems is known as predictive analytics.

Statistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[6][7]

From a theoretical viewpoint, probably approxima

In [21]:
# Serialize the  to python dictionary into a JSON formatted string
response_json_str = json.dumps(RESPONSE_JSON)

In [22]:
response_json_str

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [24]:
NUMBER=5
SUBJECT="Machine Learning"
TONE="Simple"

In [25]:
# How to setup Token Usage Tracking in LangChain

with get_openai_callback() as cb:
    response=generate_evaluate_chain({
        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": response_json_str
    })



C:\Users\Suraj Yadav\AppData\Local\Temp\ipykernel_33320\244405528.py:4: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=generate_evaluate_chain({




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

TEXT: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] The application of ML to business problems is known as predictive analytics.

Statistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on explo

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}